In [51]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import pathlib

In [52]:
from szp_funcs import get_fio, custom_create_res, print_df, load_groups, gku_inn, path_to_docs, to_double, path_to_data, load_month, gku, load_month_all

In [84]:
month = 'jun'
ped, isp, aup = load_groups('input')
out_dir = str(pathlib.Path().resolve()) + '\\200+_' + month + '\\'

In [85]:
inns = [(7717043346, 'ГАОУ ВО МГПУ'), (7723356160, 'ГБПОУ 26 КАДР'), (7734570529, 'ГАУДО Домисолька'), (7725247657, 'ГБПОУ ОКГ Столица'), (7733020995,'ГБПОУ Колледж полиции')]

In [86]:
ppl = pd.read_excel(path_to_docs + 'school_ppl_' + month + '.xlsx')
ppl = ppl[ppl.ppl > 0]

In [88]:
fail = pd.read_excel(path_to_docs + '200_ppl_' + month + '.xlsx')['snils'].to_list()

In [89]:
fails = pd.read_excel(path_to_docs + month + '_inn_fail.xlsx')['inn'].to_list()

In [90]:
jobs = pd.read_excel(path_to_docs + '200_jobs_' + month + '.xlsx')

In [91]:
res = load_month(month)

In [92]:
res = res[(res['sum_' + month] >= 54500) & (res.inn.isin(gku_inn) == False)]

In [93]:
def make_grp(row):
    if row['ppl'] > 4000:
        return 1
    if row['ppl'] < 2000:
        return 3
    return 2

In [94]:
for_tables = pd.merge(pd.merge(res[['inn', 'job_' + month, 'sum_' + month, 'snils']], jobs, how='left', left_on='job_' + month, right_on='job').drop('job', axis=1), ppl, how='left')
for_tables['grp'] = for_tables.apply(lambda row: make_grp(row), axis=1)

In [95]:
for_tables

,inn,job_apr,sum_apr,snils,job_grp,ppl,grp
0,5003021368,Помощник воспитателя,58500.00,001-171-362 75,NaN,10722.0,1
1,5003021368,Учитель,105890.88,001-191-281 85,NaN,10722.0,1
2,5003021368,Учитель,98522.15,001-244-107 65,NaN,10722.0,1
3,5003021368,Воспитатель,77794.00,001-263-234 77,NaN,10722.0,1
4,5003021368,Специалист,75815.00,001-322-594 80,Специалисты,10722.0,1
...,...,...,...,...,...,...,...
148753,9715217689,Учитель,122089.05,187-933-260 19,NaN,2488.0,2
148754,9715217689,Учитель,155611.90,191-336-708 73,NaN,2488.0,2
148755,9715217689,Педагог-организатор,110761.60,195-450-909 00,NaN,2488.0,2
148756,9715217689,Учитель,84100.05,196-998-174 73,NaN,2488.0,2


In [96]:
szp_all = for_tables[['job_grp', 'sum_' + month]].groupby(['job_grp']).mean().reset_index()
szp_all_min = for_tables[['job_grp', 'sum_' + month]].groupby(['job_grp']).min().reset_index()
szp_all_max = for_tables[['job_grp', 'sum_' + month]].groupby(['job_grp']).max().reset_index()

In [97]:
szp_all = pd.merge(szp_all, szp_all_min, how='left', on=['job_grp'])
szp_all = pd.merge(szp_all, szp_all_max, how='left', on=['job_grp'])

In [98]:
szp_all = szp_all.rename(columns={'job_'  + month:'job', 'sum_'  + month:'max', 'sum_' + month +'_x':'mean', 'sum_' + month +'_y':'min'})

In [99]:
szp_all

,job_grp,mean,min,max
0,IT - специалисты,106206.799112,54500.00,296000.00
1,Администраторы и секретари,91405.246616,54500.00,253004.04
2,Ведущие экономисты (экономисты),133255.551054,54500.00,443608.00
3,Контрактные управляющие и специалисты по закупкам,117396.353781,54902.71,230576.20
4,Научные работники,138931.039545,54550.00,289523.00
5,Руководящий состав среднего звена,118132.106814,54500.00,428900.00
6,Специалисты,97254.617865,54500.00,561736.04
7,Специалисты по кадрам и юристконсульты,114391.117469,54500.00,287625.96
8,Специалисты по обслуживанию и содержанию здани...,81510.086651,54500.00,1046209.91


In [100]:
print_df(szp_all, out_dir + 'СЗП + вилка все по группам ' + month)

In [101]:
szp = for_tables[['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index()
szp_max = for_tables[['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index()
szp_min = for_tables[['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index()
szp = pd.merge(szp, szp_min, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_max, how='left', on=['grp', 'job_grp'])
szp = szp.rename(columns={'job_'  + month:'job', 'sum_'  + month:'max', 'sum_' + month +'_x':'mean', 'sum_' + month +'_y':'min'})


In [102]:
min_good_sch = for_tables[(for_tables.inn.isin(fails) == False) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index().rename(columns={'sum_' + month: 'min_good_sch'})
max_good_sch = for_tables[(for_tables.inn.isin(fails) == False) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index().rename(columns={'sum_' + month: 'max_good_sch'})
szp_good_sch = for_tables[(for_tables.inn.isin(fails) == False) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index().rename(columns={'sum_' + month: 'szp_good_sch'})

min_bad_sch_bad_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] > 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index().rename(columns={'sum_' + month: 'min_bad_sch_bad_ppl'})
max_bad_sch_bad_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] > 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index().rename(columns={'sum_' + month: 'max_bad_sch_bad_ppl'})
szp_bad_sch_bad_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] > 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index().rename(columns={'sum_' + month: 'szp_bad_sch_bad_ppl'})

min_bad_sch_good_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index().rename(columns={'sum_' + month: 'min_bad_sch_good_ppl'})
max_bad_sch_good_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index().rename(columns={'sum_' + month: 'max_bad_sch_good_ppl'})
szp_bad_sch_good_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index().rename(columns={'sum_' + month: 'szp_bad_sch_good_ppl'})

In [103]:
szp = pd.merge(szp, min_good_sch, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, max_good_sch, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_good_sch, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, min_bad_sch_bad_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, max_bad_sch_bad_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_bad_sch_bad_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, min_bad_sch_good_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, max_bad_sch_good_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_bad_sch_good_ppl, how='left', on=['grp', 'job_grp'])

In [104]:
print_df(szp, out_dir + 'СЗП + вилка все по контингенту ' + month)

In [105]:
for sch in inns:
    szp_t = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).mean().reset_index()
    szp_t_min = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).min().reset_index()
    szp_t_max = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).max().reset_index()
    szp_t_cnt = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).count().reset_index().rename(columns={'sum_' + month: 'cnt'})
    szp_t = pd.merge(szp_t, szp_t_min, how='left', on=['job_grp'])
    szp_t = pd.merge(szp_t, szp_t_max, how='left', on=['job_grp'])
    szp_t = pd.merge(szp_t, szp_t_cnt, how='left', on=['job_grp'])
    szp_t = szp_t.rename(columns={'job_'  + month:'job', 'sum_'  + month:'max', 'sum_' + month +'_x':'mean', 'sum_' + month +'_y':'min'})
    print_df(szp_t, out_dir + 'Вилка организации 200+ ' + month + ' ' + sch[1])